# TP3 Audionumérique 3SN-M (2024)


**NOM1** : LAFARGUE

**Prénom1** : Héloïse

**Email1** : hls.lafargue@gmail.com

Ce document est en lecture seule, veillez l'enregistrer chez vous pour pouvoir le modifier.

L'objectif de cette séance est :
1.   D'utiliser des modèles pré-appris sur un corpus de mots de commandes.
2.   D'évaluer les performances de deux systèmes.

Vous avez une semaine pour déposer le colab.

Sur ce TP, il est préférable d'activer une session utilisant un GPU. Dans le menu  "Execution" -> "Modifier le type d'éxécution" vérifier que GPU soit sélectionné.

In [ ]:
!apt-get install -qq bc tree sox
!pip install speechbrain transformers

Selecting previously unselected package bc.
(Reading database ... 121671 files and directories currently installed.)
Preparing to unpack .../0-bc_1.07.1-3build1_amd64.deb ...
Unpacking bc (1.07.1-3build1) ...
Selecting previously unselected package libopencore-amrnb0:amd64.
Preparing to unpack .../1-libopencore-amrnb0_0.1.5-1_amd64.deb ...
Unpacking libopencore-amrnb0:amd64 (0.1.5-1) ...
Selecting previously unselected package libopencore-amrwb0:amd64.
Preparing to unpack .../2-libopencore-amrwb0_0.1.5-1_amd64.deb ...
Unpacking libopencore-amrwb0:amd64 (0.1.5-1) ...
Selecting previously unselected package libsox3:amd64.
Preparing to unpack .../3-libsox3_14.4.2+git20190427-2+deb11u2ubuntu0.22.04.1_amd64.deb ...
Unpacking libsox3:amd64 (14.4.2+git20190427-2+deb11u2ubuntu0.22.04.1) ...
Selecting previously unselected package libsox-fmt-alsa:amd64.
Preparing to unpack .../4-libsox-fmt-alsa_14.4.2+git20190427-2+deb11u2ubuntu0.22.04.1_amd64.deb ...
Unpacking libsox-fmt-alsa:amd64 (14.4.2+git

## Préparation des données

### Packages

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import librosa
import os

import IPython
from librosa.display import specshow

*Optionnel* Si vous souhaitez monter le drive de votre compte google, exécutez la commande ci-dessous (ça vous demandera d'ouvrir une page et vous connecter avec votre compte Google et copier le code ci-dessous). Ca vous permettra d'accéder à votre drive avec le chemin suivant : "/content/gdrive/My Drive/".

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


### Dataset brut (audio)

In [ ]:
#télécharge Google Speech Commands Dataset (version 0.02)
!wget http://download.tensorflow.org/data/speech_commands_v0.02.tar.gz

# décompactage du jeu de données dans le dossier data/
!mkdir data
!tar xzf speech_commands_v0.02.tar.gz -C data

--2024-01-28 22:58:36--  http://download.tensorflow.org/data/speech_commands_v0.02.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 142.251.172.207, 173.194.206.207, 108.177.112.207, ...
Connecting to download.tensorflow.org (download.tensorflow.org)|142.251.172.207|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2428923189 (2.3G) [application/gzip]
Saving to: ‘speech_commands_v0.02.tar.gz’

speech_commands_v0. 100%[===================>]   2.26G   168MB/s    in 18s     

2024-01-28 22:58:54 (130 MB/s) - ‘speech_commands_v0.02.tar.gz’ saved [2428923189/2428923189]



## Ecoute de fichiers

In [ ]:
#Ecouter un audio :
path = "/content/data/cat/030ec18b_nohash_2.wav" #@param {type:"string"}

IPython.display.Audio(path)

## Constitution du corpus de test

La liste des fichiers à utiliser pour le test est contenu dans le fichier "/content/data/testing_list.txt".

Vous allez devoir mettre en place la reconnaissance pour les deux systèmes ASR le calcul des résultats sur cette liste de fichiers .wav.

Plus de détails sur le nommage des fichiers ici :
"/content/data/README.md"

In [ ]:
import random
from sklearn.metrics import confusion_matrix
import seaborn as sns
from collections import defaultdict

## Mise en place d'un modèle pré-appris

Modèle récupéré ici :
https://huggingface.co/speechbrain/asr-crdnn-rnnlm-librispeech


In [ ]:
from speechbrain.pretrained import EncoderDecoderASR

asr_model = EncoderDecoderASR.from_hparams(source="speechbrain/asr-crdnn-rnnlm-librispeech", savedir="pretrained_model", run_opts={"device": "cuda"})  # Ensure model uses the GPU

audio_file = '/content/data/cat/030ec18b_nohash_2.wav'
asr_model.transcribe_file(audio_file)


'CAT'

In [ ]:
asr_model.transcribe_file("/content/data/right/bb05582b_nohash_3.wav")

'WRIGHT'

In [ ]:
# Lire le fichier de test et organiser par classe
with open('/content/data/testing_list.txt', 'r') as file:
    test_files = file.readlines()

class_files = defaultdict(list)
for audio_file in test_files:
    label = audio_file.split('/')[0].lower()  # Convertir en minuscule
    class_files[label].append(audio_file.strip())

# Sélectionner 20 exemples pour chaque classe
#selected_files = []
#for label in class_files:
    #selected_files.extend(random.sample(class_files[label], min(20, len(class_files[label]))))

# Prédiction et évaluation
total_correct = 0
total_predictions = 0
class_correct = defaultdict(int)
class_total = defaultdict(int)

for audio_file in test_files:
    label = audio_file.split('/')[0].lower()
    predicted_label = asr_model.transcribe_file('/content/data/' + audio_file.strip())
    is_correct = label == predicted_label.lower()
    total_correct += is_correct
    total_predictions += 1
    class_correct[label] += is_correct
    class_total[label] += 1
    print("Label :", label, " - Prediction :", predicted_label, " - Correct:", is_correct)


Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
Label : zero  - Prediction : DAWN  - Correct: False
Label : zero  - Prediction : BEARER  - Correct: False
Label : zero  - Prediction : BED  - Correct: False
Label : zero  - Prediction : ZERO  - Correct: True
Label : zero  - Prediction : NINE  - Correct: False
Label : zero  - Prediction : TRAPPED  - Correct: False
Label : zero  - Prediction : ABSOLUTELY  - Correct: False
Label : zero  - Prediction : STOP  - Correct: False
Label : zero  - Prediction : NONE  - Correct: False
Label : zero  - Prediction : CREEP  - Correct: False
Label : zero  - Prediction : FULLER  - Correct: False
Label : zero  - Prediction : DAWN  - Correct: False
Label : zero  - Prediction : BIRD  - Correct: False
Label : zero  - Prediction : SECT  - Correct: False
Label : zero  - Prediction : FOLLOW  - Correct: False
Label : zero  - Prediction : ZERO  - Correct: True
Label : zero  - Prediction : DOG  - Correct: False
Label : zero  - Prediction

In [ ]:
# Calcul de l'accuracy totale
total_accuracy = total_correct / total_predictions if total_predictions > 0 else 0

# Calcul de l'accuracy par classe
accuracy_per_class = {label: class_correct[label] / class_total[label] if class_total[label] > 0 else 0 for label in class_files}

print("Accuracy Totale:", total_accuracy)
print("Accuracy par Classe:", accuracy_per_class)

Accuracy Totale: 0.03534756928668787
Accuracy par Classe: {'right': 0.0, 'eight': 0.031862745098039214, 'cat': 0.08762886597938144, 'tree': 0.10880829015544041, 'backward': 0.01818181818181818, 'learn': 0.0, 'bed': 0.0966183574879227, 'happy': 0.10344827586206896, 'go': 0.05223880597014925, 'dog': 0.10909090909090909, 'no': 0.05432098765432099, 'wow': 0.03398058252427184, 'follow': 0.011627906976744186, 'nine': 0.049019607843137254, 'left': 0.0024271844660194173, 'stop': 0.04866180048661801, 'three': 0.04691358024691358, 'sheila': 0.0660377358490566, 'one': 0.03759398496240601, 'bird': 0.07567567567567568, 'zero': 0.05263157894736842, 'seven': 0.014778325123152709, 'up': 0.042352941176470586, 'visual': 0.030303030303030304, 'marvin': 0.035897435897435895, 'two': 0.05188679245283019, 'house': 0.010471204188481676, 'down': 0.0049261083743842365, 'six': 0.005076142131979695, 'yes': 0.03341288782816229, 'on': 0.010101010101010102, 'five': 0.015730337078651686, 'forward': 0.0129032258064516

## Mise en place d'un deuxième modèle pré-appris
A vous maintenant d'utiliser un sytème cléf en main pour réaliser de la reconnaissance de la parole (ASR : Automatic Speech recognition) en anglais.
Essayez par exemple un système E2E récent.

https://huggingface.co/openai/whisper-small

In [ ]:
!pip install transformers torch
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [ ]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from collections import defaultdict
import torchaudio
import torch
import string

# Charger le modèle et le processeur Whisper
processor = WhisperProcessor.from_pretrained("openai/whisper-small")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")
forced_decoder_ids = processor.get_decoder_prompt_ids(language="english", task="transcribe")

# Lire le fichier de test et organiser par classe
test_files = []
with open('/content/data/testing_list.txt', 'r') as file:
    test_files = file.readlines()

class_files = defaultdict(list)
for audio_file in test_files:
    label = audio_file.split('/')[0].lower()
    class_files[label].append('/content/data/' + audio_file.strip())

# Sélectionner 10% des exemples pour chaque classe
selected_files = defaultdict(list)
for label, files in class_files.items():
    num_files = max(1, len(files) // 10)  # Calculer 10% de la taille de la classe
    selected_files[label] = random.sample(files, num_files)

# Prédiction et évaluation
total_correct = 0
total_predictions = 0
class_correct = defaultdict(int)
class_total = defaultdict(int)

# Transcrire chaque fichier audio et évaluer
for label, files in selected_files.items():
    for file_path in files:
        waveform, sample_rate = torchaudio.load(file_path)

        # Assurer le taux d'échantillonnage correct
        if sample_rate != 16000:
            waveform = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=16000)(waveform)
            sample_rate = 16000

        # Préparation des caractéristiques d'entrée
        input_features = processor(waveform.squeeze().numpy(), sampling_rate=sample_rate, return_tensors="pt").input_features

        # Générer des identifiants de token
        predicted_ids = model.generate(input_features, forced_decoder_ids=forced_decoder_ids)

        # Décoder les identifiants de token en texte
        predicted_label = processor.batch_decode(predicted_ids, skip_special_tokens=True)[0].lower()
        # Nettoyer la transcription en supprimant les points
        #predicted_label = predicted_label.replace('.', '').strip()
        # Nettoyer la transcription en supprimant toutes les ponctuations
        translator = str.maketrans('', '', string.punctuation)
        predicted_label = predicted_label.translate(translator).strip()

        is_correct = label == predicted_label

        # Mise à jour des compteurs
        total_correct += is_correct
        total_predictions += 1
        class_correct[label] += is_correct
        class_total[label] += 1

        print("Label :", label, " - Prediction :", predicted_label, " - Correct:", is_correct)

Label : right  - Prediction : right  - Correct: True
Label : right  - Prediction : right  - Correct: True
Label : right  - Prediction : right  - Correct: True
Label : right  - Prediction : right  - Correct: True
Label : right  - Prediction : right  - Correct: True
Label : right  - Prediction : right  - Correct: True
Label : right  - Prediction : right  - Correct: True
Label : right  - Prediction : right  - Correct: True
Label : right  - Prediction : right  - Correct: True
Label : right  - Prediction : right  - Correct: True
Label : right  - Prediction : right  - Correct: True
Label : right  - Prediction : right  - Correct: True
Label : right  - Prediction : right  - Correct: True
Label : right  - Prediction : right  - Correct: True
Label : right  - Prediction : thank you  - Correct: False
Label : right  - Prediction : right  - Correct: True
Label : right  - Prediction : right  - Correct: True
Label : right  - Prediction : right  - Correct: True
Label : right  - Prediction : right  - Co

In [ ]:
# Calcul de l'accuracy
total_accuracy = total_correct / total_predictions if total_predictions > 0 else 0
accuracy_per_class = {label: class_correct[label] / class_total[label] if class_total[label] > 0 else 0 for label in class_correct}

print("Accuracy Totale:", total_accuracy)
print("Accuracy par Classe:", accuracy_per_class)

Accuracy Totale: 0.7695852534562212
Accuracy par Classe: {'right': 0.9487179487179487, 'eight': 0.15, 'cat': 0.8421052631578947, 'tree': 0.8421052631578947, 'backward': 0.8125, 'learn': 0.625, 'bed': 0.6, 'happy': 0.95, 'go': 0.975, 'dog': 0.8636363636363636, 'no': 1.0, 'wow': 0.85, 'follow': 0.8823529411764706, 'nine': 0.475, 'left': 0.8292682926829268, 'stop': 0.975609756097561, 'three': 0.725, 'sheila': 0.8571428571428571, 'one': 0.8205128205128205, 'bird': 0.7777777777777778, 'zero': 0.7317073170731707, 'seven': 0.8, 'up': 0.8809523809523809, 'visual': 0.5, 'marvin': 0.8947368421052632, 'two': 0.6190476190476191, 'house': 0.8421052631578947, 'down': 0.8, 'six': 0.5641025641025641, 'yes': 0.926829268292683, 'on': 0.8205128205128205, 'five': 0.75, 'forward': 0.8, 'off': 0.725, 'four': 0.65}


## Questions complémentaires

1.  Quel est le corpus utilisé ? Comment a-t-il été constitué ?
2.  Quelle est l'architecture du modèle 1 ? (type de réseau de neurones profonds, présence d'un modèle de langage, principe de fonctionnement des différents blocs enchaines, corpus pour apprendre l'acoustique, corpus pour apprendre le modèle de langage)
2.  Quelle est l'architecture du modèle 2 ? (type de réseau de neurones profonds, présence d'un modèle de langage, principe de fonctionnement des différents blocs enchaines, corpus pour apprendre l'acoustique, corpus pour apprendre le modèle de langage)
3.  Quel est le système qui présente les meilleurs performances ?
4.  Est-ce que les erreurs les plus fréquentes peuvent s'expliquer ? En particulier celles du système 1.
5. Quelles sont les performances de l'état de l'art sur ce corpus Speech Commands ?

Vos réponses....

#### **1. Corpus - Google Speech Commands Dataset (Version 0.02)**
- Le corpus Google Speech Commands Dataset est un corpus riche et diversifié spécialement conçu pour la reconnaissance de commandes vocales simples et courtes. Sa variété de mots, de locuteurs et d'accents en fait un choix populaire pour entraîner et tester des modèles de reconnaissance vocale.
- Le dataset contient des milliers d'enregistrements audio courts (d'1 seconde environ) de 35 mots différents. Ces mots incluent des chiffres, des commandes simples comme "yes", "no", "up", "down", et quelques noms.
- Les enregistrements ont été fournis par une large gamme de participants à travers une interface en ligne. Cela a permis de collecter une diversité de voix, d'accents et de prononciations.
- Les fichiers audio sont généralement au format WAV et sont enregistrés à un taux d'échantillonnage standard pour l'ASR, comme 16 kHz. Ils sont organisés dans des dossiers portant le nom du mot prononcé.
- `testing_list.txt` contient les chemins des fichiers audio spécifiques à utiliser pour les tests. Cela assure la cohérence dans l'évaluation des modèles ASR.
- Le fichier `README.md` fournit des informations supplémentaires sur le dataset, y compris sa structure, le nommage des fichiers et les instructions pour l'utiliser.


#### **2. Modèle 1**

Le modèle 1, `speechbrain/asr-crdnn-rnnlm-librispeech`, est un modèle de reconnaissance automatique de la parole (ASR) développé par SpeechBrain. Voici les détails de son architecture et de ses composants :

**Type de Réseau de Neurones Profonds : CRDNN (Convolutional, Recurrent, and Dense Neural Networks)**
   - **Réseau Convolutif (CNN)** : Cette partie traite les caractéristiques acoustiques en entrée. Les CNN sont efficaces pour capturer les motifs locaux et les invariances temporelles et fréquentielles dans les données audio.
   - **Réseau Récurrent (RNN)** : Après le traitement par le CNN, les RNN (typiquement des LSTM ou GRU) sont utilisés pour capturer les dépendances temporelles dans les données. Ils sont cruciaux pour modéliser la séquence temporelle des paroles.
   - **DNN (Dense Neural Networks)** : Les réseaux denses ajoutent des couches entièrement connectées pour la transformation non-linéaire des caractéristiques, contribuant à une meilleure discrimination des différents phonèmes ou mots.

**Présence d'un Modèle de Langage : Modèle de Langage à Réseau Neuronal Récurrent (RNNLM)**
   - Ce modèle de langage est généralement entraîné séparément et est utilisé pour aider à la prédiction des mots en fournissant un contexte linguistique. Il améliore la précision en tenant compte des probabilités des séquences de mots, ce qui est particulièrement utile pour les mots homophones ou dans des contextes bruités.

**Principe de Fonctionnement des Différents Blocs Enchaînés : Flux de Traitement**
   - Les données audio sont d'abord traitées par le CNN pour extraire des caractéristiques locales.
   - Les RNN traitent ensuite ces caractéristiques pour modéliser la séquence temporelle.
   - Les réseaux denses effectuent une transformation supplémentaire avant la prédiction des phonèmes ou des mots.
   - Le modèle de langage RNNLM est appliqué en post-traitement pour affiner les prédictions en tenant compte du contexte linguistique.

**Corpus pour Apprendre l'Acoustique : Corpus LibriSpeech**
   - Ce modèle est généralement entraîné sur le corpus LibriSpeech, qui est un large ensemble de données de discours en anglais, dérivé de livres audio. Ce corpus est diversifié en termes d'accents, de styles de parole et de qualité d'enregistrement, ce qui le rend idéal pour l'entraînement de modèles ASR robustes.

**Corpus pour Apprendre le Modèle de Langage : Corpus pour le RNNLM**
   - Le modèle de langage RNNLM est souvent entraîné sur un large corpus textuel pour bien comprendre la structure et la séquence des mots en langue naturelle. Le corpus spécifique dépend de la mise en œuvre, mais il comprend généralement un grand ensemble de textes en anglais provenant de diverses sources pour garantir la diversité et la richesse linguistique.

En résumé, le modèle 1 combine des réseaux convolutifs, récurrents et denses pour traiter les caractéristiques acoustiques, tandis qu'un modèle de langage RNNLM est utilisé pour améliorer la prédiction des mots en se basant sur le contexte linguistique. Le modèle est entraîné sur le corpus LibriSpeech pour l'acoustique et un corpus textuel étendu pour le modèle de langage.

#### 3. Modèle 2

Le modèle Whisper d'OpenAI est un système de reconnaissance automatique de la parole (ASR) basé sur des réseaux de neurones profonds. Voici une vue d'ensemble de son architecture et de ses composants :

**Type de Réseau de Neurones Profonds**

1. **Architecture Transformer**: Whisper utilise une architecture basée sur les Transformers, similaire à celle utilisée dans des modèles de langage comme GPT et BERT. Les Transformers sont particulièrement efficaces pour traiter des séquences de données, comme le texte ou l'audio, grâce à leur capacité à gérer des dépendances à long terme dans les données.

2. **Encodage Audio**: Le modèle commence par transformer les entrées audio en représentations utiles pour la tâche de reconnaissance de la parole. Cela implique généralement des étapes de prétraitement et de codage pour convertir les ondes sonores en une forme qui peut être traitée par le réseau de neurones.

**Présence d'un Modèle de Langage**

Whisper intègre un modèle de langage dans son architecture. Cela lui permet non seulement de reconnaître les mots prononcés mais aussi de comprendre le contexte pour améliorer la précision de la transcription. Le modèle de langage aide à prédire le mot suivant dans une séquence, en tenant compte des mots précédents.

**Principe de Fonctionnement des Différents Blocs Enchaînés**

1. **Traitement Acoustique**: Le son est d'abord traité pour extraire des caractéristiques pertinentes. Cela peut inclure la conversion de l'audio brut en spectrogrammes ou en d'autres représentations qui capturent les aspects essentiels du signal audio.

2. **Encodage par le Transformer**: Les représentations extraites sont ensuite passées à travers les couches de Transformer. Ces couches apprennent à comprendre à la fois le contenu acoustique et linguistique du discours.

3. **Décodage et Prédiction**: Le modèle produit ensuite des transcriptions sous forme de texte. Cette étape implique souvent la traduction des représentations internes en mots ou phrases dans une langue spécifique, en utilisant à la fois la compréhension acoustique et le contexte linguistique.


**Corpus Acoustique** :
Pour apprendre à reconnaître la parole, Whisper aurait été entraîné sur un large éventail de données acoustiques, comprenant des échantillons de discours dans diverses langues, accents, et contextes acoustiques. Cela comprend des enregistrements de discours dans des environnements naturels, des podcasts, des vidéos, etc.

**Corpus de Modèle de Langage** :
Le modèle de langage de Whisper est probablement entraîné sur un large corpus textuel, incluant des livres, des articles, des transcriptions de discours, et d'autres sources textuelles. Cela permet au modèle de comprendre et de générer du texte qui est cohérent et grammaticalement correct.

En résumé, Whisper combine une architecture Transformer avancée avec un entraînement approfondi sur des données audio et linguistiques variées, lui permettant de réaliser une transcription précise et contextuellement informée de la parole en texte.

#### 4. Meilleures performances

Sachant que ces résultats ont été obtenus en entraînant le modèle 1 sur le corpus complet et le modèle 2 sur seulement 10% de chaque classe du corpus (car c'était très long), voici les résultats :

Le modèle 2 a une précision de 76.96%, ce qui est très supérieur à la précision totale du modèle 1, qui n'est que de 3.53%. De plus, en examinant la précision par classe pour chaque modèle, le modèle 2 montre des taux de précision élevés pour presque toutes les classes, avec plusieurs classes ayant des taux de précision supérieurs à 80%, et même quelques-unes atteignant 100%. En comparaison, la précision par classe du modèle 1 est nettement inférieure, avec plusieurs classes ayant des taux de précision très bas ou nuls.

Exemple de mots très bien reconnus par le modèle 2 :
"right" : 94.87%, "happy" : 95%, "go" : 97.5%, "no" : 100%, "stop" : 97.56%,   "up" : 88.10%, "marvin" : 89.47%, "yes" : 92.68% et "on" : 82.05%.

En conclusion, sur la base de ces données, le modèle 2 (le système E2E "openai/whisper-small") est nettement supérieur en termes de performance par rapport au modèle 1 pour le corpus Google Speech Commands Dataset (Version 0.02).

#### 5. Erreurs fréquentes

Résultats pour le modèle 1 :
  - Faible Précision Globale (3.53%): Indique que le modèle a généralement du mal à reconnaître les mots dans ce corpus. Cela pourrait être dû à un désalignement entre les caractéristiques du corpus d'entraînement et celles de Speech Commands.

  - Classes avec 0% de Précision (ex. "right", "learn"): Cela suggère que ces mots n'ont été correctement identifiés dans aucun cas.  Ces mots sont plus complexes phonétiquement. Par exemple, "right" combine une consonne initiale fricative, une voyelle diphtongue, et une consonne finale. "Learn" combine une consonne liquide avec une rime nasale, ce qui peut être difficile à distinguer. Ces mots peuvent être facilement confondus avec d'autres mots ayant des sons similaires, surtout dans un contexte où le modèle ne bénéficie pas de signaux contextuels forts.
  

  - Meilleures Performances sur Certains Mots (ex. "dog", "tree"): Ces mots ont une précision relativement meilleure, en effet, ils ont des caractéristiques phonétiques distinctes. Par exemple, "dog" a un son de voyelle fort et un son final guttural (le "g"), tandis que "tree" a une consonne initiale fricative suivie d'une voyelle claire. Ces caractéristiques peuvent être plus facilement distinguées par le modèle. Et ils sont relativement simples en termes de structure phonétique, ce qui réduit la probabilité d'erreurs.

Pour le modèle 2 :  Moins bonnes performances pour "eight" : 15%, "nine" : 47.5% et "visual" : 50%.

#### 6. Etat de l'art sur le Corpus Speech Command

Le corpus de Speech Commands est étudié dans l'article "Speech Commands: A Dataset for Limited-Vocabulary Speech Recognition" de Pete Warden, publié en avril 2018 (https://arxiv.org/abs/1804.03209). Cet ensemble de données est destiné à faciliter l'entraînement et le test de petits modèles capables de détecter lorsqu'un seul mot est prononcé parmi un ensemble de dix mots ou moins, avec un minimum de faux positifs dus au bruit de fond ou à des paroles non pertinentes.

Voici quelques caractéristiques :

- Taille et diversité : Il contient 105,829 enregistrements de 2,618 locuteurs, offrant une bonne diversité en termes d'accents et de prononciations.

- Vocabulaire limité : Il se concentre sur des mots-clés spécifiques, rendant le corpus adapté à la reconnaissance de commandes vocales simples.

- Utilisation dans la recherche : Il a servi de base pour le développement et l'évaluation de modèles de reconnaissance de la parole, en particulier pour les appareils à faible consommation.

L'évaluation historique du corpus "Speech Commands" montre une amélioration notable des performances avec le temps. La première version du dataset, sortie en août 2017, a permis d'atteindre une précision de 85,4% en utilisant le modèle convolutionnel par défaut de TensorFlow. Avec la deuxième version du dataset, la précision s'est améliorée, atteignant 88,2% sur l'ensemble de test de la V2 et 89,7% sur celui de la V1. Cela indique que les données de la V2 ont bien amélioré la précision par rapport à la V1.